In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
filepath=[r"B:\project\human\human_chunk_7.csv",r"B:\project\ai\ai_chunk_7.csv","B:\project\human\human_chunk_8.csv",r"B:\project\ai\ai_chunk_8.csv","B:\project\human\human_chunk_9.csv",r"B:\project\ai\ai_chunk_9.csv","B:\project\human\human_chunk_10.csv",r"B:\project\ai\ai_chunk_10.csv"]
dfs = [pd.read_csv(filepath) for filepath in filepath]
df = pd.concat(dfs, ignore_index=True)
print(df.shape)


(122320, 2)


In [2]:
from transformers import LongformerTokenizer
data=df['text'].tolist()
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
tokens=tokenizer(data,return_tensors='pt',padding=True,truncation=True)

In [3]:
from sklearn.model_selection import train_test_split
x=tokens['input_ids']
y=df['generated']
xtrain,xtemp,ytrain,ytemp=train_test_split(x,y,test_size=0.3,random_state=42)
xtest,xval,ytest,yval=train_test_split(xtemp,ytemp,test_size=0.3,random_state=42)
xtest,xtrain,xval=xtest.numpy(),xtrain.numpy(),xval.numpy()

In [4]:
from keras.models import load_model
model=load_model('finetuned2.keras')
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain,ytrain,epochs=5,batch_size=256,validation_data=(xval,yval))



Epoch 1/5


335/335 [==============================] - 231s 684ms/step - loss: 0.0072 - accuracy: 0.9979 - val_loss: 0.0049 - val_accuracy: 0.9987
Epoch 2/5
335/335 [==============================] - 231s 689ms/step - loss: 0.0016 - accuracy: 0.9996 - val_loss: 0.0060 - val_accuracy: 0.9988
Epoch 3/5
335/335 [==============================] - 234s 698ms/step - loss: 0.0011 - accuracy: 0.9997 - val_loss: 0.0044 - val_accuracy: 0.9990
Epoch 4/5
335/335 [==============================] - 233s 697ms/step - loss: 7.6448e-04 - accuracy: 0.9999 - val_loss: 0.0049 - val_accuracy: 0.9987
Epoch 5/5
335/335 [==============================] - 235s 701ms/step - loss: 4.6908e-04 - accuracy: 0.9999 - val_loss: 0.0058 - val_accuracy: 0.9986


In [5]:
model.evaluate(xtest,ytest)
model.save('finetuned3.keras')

803/803 [==============================] - 14s 17ms/step - loss: 0.0071 - accuracy: 0.9987
